#### Loading and preparing the dataset

In [1]:
from datasets import load_dataset

In [2]:
# load my recently created dataset from HuggingFace Datasets hub
issues_dataset = load_dataset("rajknakka/github-issues-comments", split="train")
issues_dataset

Found cached dataset parquet (C:/Users/Raj/.cache/huggingface/datasets/rajknakka___parquet/rajknakka--github-issues-comments-d10cf254d383122f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'body', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'draft', 'pull_request', 'is_pull_request'],
    num_rows: 4900
})

In [3]:
# filter out pull requests as these tend to be rarely used for answering user queries and will introduce noise in our search engine
issues_dataset = issues_dataset.filter(lambda row: row['is_pull_request'] == False and len(row["comments"]) > 0)
issues_dataset

Loading cached processed dataset at C:\Users\Raj\.cache\huggingface\datasets\rajknakka___parquet\rajknakka--github-issues-comments-d10cf254d383122f\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-be07580daa56c5b4.arrow


Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'body', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'draft', 'pull_request', 'is_pull_request'],
    num_rows: 1598
})

In [4]:
# keep the title, body, comments and html_url columns and remove the rest of the columns
columns_to_keep = ["title", "body", "comments", "html_url"]
columns = issues_dataset.column_names
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
prepared_dataset = issues_dataset.remove_columns(columns_to_remove)
prepared_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 1598
})

In [5]:
prepared_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 1598
})

#### Create embeddings with the comments and their context

In [6]:
# first conver the dataset to a pandas dataframe
prepared_dataset.set_format("pandas")
issues_df = prepared_dataset[:]

In [7]:
# inspect the dataframe to see the four columns
issues_df["comments"][1].tolist()

['Hi ! The audio data don\'t always exist as files on disk - the blobs are often stored in the Arrow files. For now I\'d suggest disabling decoding with `.cast_column("audio", Audio(decode=False))` and apply your own decoding that handles corrupted files (maybe to filter them out ?)\r\n\r\ncc @sanchit-gandhi since it\'s related to our discussion about allowing users to make decoding return `None` and show a warning when there are corrupted files',
 'Thanks @lhoestq, I wasn\'t aware of the decode flag. It makes more sense as you say to show a warning when there are corrupted files together with some metadata of the file that allows to filter them from the dataset.\r\n\r\nMy workaround was to catch the LibsndfileError and generate a dummy audio with an unsual sample rate to filter it later. However returning `None` seems better. \r\n\r\n`try:\r\n    array, sampling_rate = sf.read(file)\r\nexcept sf.LibsndfileError:\r\n    print("bad file")\r\n    array = np.array([0.0])\r\n    sampling_r

In [8]:
# explode the dataframe to have one row per comment
comments_df = issues_df.explode("comments", ignore_index=True)
comments_df.head(4)

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues...,Support for data with instance-wise dictionary...,Hi ! We use the Arrow columnar format under th...,### Feature request\n\nI notice that when load...
1,https://github.com/huggingface/datasets/issues...,Return the audio filename when decoding fails ...,Hi ! The audio data don't always exist as file...,### Feature request\r\n\r\nReturn the audio fi...
2,https://github.com/huggingface/datasets/issues...,Return the audio filename when decoding fails ...,"Thanks @lhoestq, I wasn't aware of the decode ...",### Feature request\r\n\r\nReturn the audio fi...
3,https://github.com/huggingface/datasets/issues...,IndexError Not Solving -> IndexError: Invalid ...,https://colab.research.google.com/#scrollTo=AQ...,### Describe the bug\n\nin <cell line: 1>:1 ...


In [9]:
# check for any null values in any of the columns of the dataframe
comments_df.isnull().sum()


html_url    0
title       0
comments    0
body        6
dtype: int64

In [10]:
# remove any rows with null values
comments_df = comments_df.dropna()
comments_df

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues...,Support for data with instance-wise dictionary...,Hi ! We use the Arrow columnar format under th...,### Feature request\n\nI notice that when load...
1,https://github.com/huggingface/datasets/issues...,Return the audio filename when decoding fails ...,Hi ! The audio data don't always exist as file...,### Feature request\r\n\r\nReturn the audio fi...
2,https://github.com/huggingface/datasets/issues...,Return the audio filename when decoding fails ...,"Thanks @lhoestq, I wasn't aware of the decode ...",### Feature request\r\n\r\nReturn the audio fi...
3,https://github.com/huggingface/datasets/issues...,IndexError Not Solving -> IndexError: Invalid ...,https://colab.research.google.com/#scrollTo=AQ...,### Describe the bug\n\nin <cell line: 1>:1 ...
4,https://github.com/huggingface/datasets/issues...,IndexError Not Solving -> IndexError: Invalid ...,Looks related to https://discuss.huggingface.c...,### Describe the bug\n\nin <cell line: 1>:1 ...
...,...,...,...,...
5645,https://github.com/huggingface/datasets/issues...,making sure datasets are not loaded in memory ...,Hi! You can use the `assert not bool(dataset....,Hi\r\nI am dealing with large-scale datasets w...
5646,https://github.com/huggingface/datasets/issues...,sample multiple datasets,here I share my dataloader currently for multi...,"Hi\r\nI am dealing with multiple datasets, I n..."
5647,https://github.com/huggingface/datasets/issues...,sample multiple datasets,Hi @rabeehkarimimahabadi any luck with updatin...,"Hi\r\nI am dealing with multiple datasets, I n..."
5648,https://github.com/huggingface/datasets/issues...,sample multiple datasets,"Hi @pushkalkatara yes I solved it back then, h...","Hi\r\nI am dealing with multiple datasets, I n..."


In [11]:
# check for any null values in any of the columns of the dataframe
comments_df.isnull().sum()

html_url    0
title       0
comments    0
body        0
dtype: int64

In [12]:
# done with pandas dataframe, convert back to HuggingFace dataset
from datasets import Dataset

comments_dataset = Dataset.from_pandas(comments_df)
comments_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body', '__index_level_0__'],
    num_rows: 5644
})

In [13]:
# create a new comments_length column to store the number of words per comment
comments_dataset = comments_dataset.map(lambda row: {"comments_length": len(row["comments"].split())})

Map:   0%|          | 0/5644 [00:00<?, ? examples/s]

In [14]:
# use the new column to filter out comments with less than 15 words
comments_dataset = comments_dataset.filter(lambda row: row["comments_length"] > 15)
comments_dataset

Filter:   0%|          | 0/5644 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', '__index_level_0__', 'comments_length'],
    num_rows: 4171
})

In [15]:
# define a function to concatenate the title, body and comments columns into a single text column
def concatenate_columns(row):
    return {"text": row["title"] + " \n " + row["body"] + " \n " + row["comments"]}

In [16]:
comments_dataset = comments_dataset.map(concatenate_columns)
comments_dataset

Map:   0%|          | 0/4171 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', '__index_level_0__', 'comments_length', 'text'],
    num_rows: 4171
})

#### Create text embeddings

In [17]:
# we will use the sentence-transformers library to encode our text into vectors
# since we are using asymetric semantic search, short query whose answer we would like to find in a longer document
# we will use the multi-qa-mpnet-base-dot-vi model which is trained on the Natural Questions dataset and has the best performance for semantic search

from transformers import AutoTokenizer, AutoModel

# load the tokenizer with the same checkpoint as the model
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [18]:
# speed up the embedding process by using the GPU
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [19]:
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [20]:
# define a CLS pooling function on our model's output to get a single vector representation of the text
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [21]:
# define a function to tokenize our text and convert it to tensors. Place tensors on the GPU, feed them to the model and pool the output
def get_embeddings(text_list):
    encoded_input = tokenizer(text_list, padding=True, truncation=True, return_tensors="pt")
    encoded_input = {key: value.to(device) for key, value in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [22]:
# test the function on a sample text from our corpus and inspect the output shape
embedding = get_embeddings(comments_dataset["text"][0])
embedding.shape

torch.Size([1, 768])

In [23]:
# use dataset.map() to apply the function to all the text in our dataset and put that in a new embeddings column
# we will also convert the embeddings to numpy arrays because datasets can't store tensors and requires numpy arrays to index the embeddings with FAISS
embeddings_dataset = comments_dataset.map(lambda row: {"embeddings": get_embeddings(row["text"]).detach().cpu().numpy()[0]})

Map:   0%|          | 0/4171 [00:00<?, ? examples/s]

#### Use FAISS for efficient similarity search

In [24]:
! pip install faiss-cpu

In [25]:
# create a FAISS index and add the embeddings to it
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/5 [00:00<?, ?it/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', '__index_level_0__', 'comments_length', 'text', 'embeddings'],
    num_rows: 4171
})

In [39]:
# perform a semantic search on the index with a query
query = "How can I load a dataset offline?"
query_embedding = get_embeddings([query]).cpu().detach().numpy()
query_embedding.shape

(1, 768)

In [40]:
# perform a similarity search on the index with the query embedding
scores, retrieved_examples = embeddings_dataset.get_nearest_examples("embeddings", query_embedding, k=5)

In [32]:
scores

array([41.922493, 41.922493, 42.686325, 43.964485, 43.964485],
      dtype=float32)

In [34]:
retrieved_examples

{'html_url': ['https://github.com/huggingface/datasets/issues/2532',
  'https://github.com/huggingface/datasets/issues/2532',
  'https://github.com/huggingface/datasets/issues/1167',
  'https://github.com/huggingface/datasets/issues/1830',
  'https://github.com/huggingface/datasets/issues/1830'],
 'title': ["Tokenizer's normalization preprocessor cause misalignment in return_offsets_mapping for tokenizer classification task",
  "Tokenizer's normalization preprocessor cause misalignment in return_offsets_mapping for tokenizer classification task",
  '❓ On-the-fly tokenization with datasets, tokenizers, and torch Datasets and Dataloaders',
  'using map on loaded Tokenizer 10x - 100x slower than default Tokenizer?',
  'using map on loaded Tokenizer 10x - 100x slower than default Tokenizer?'],
 'comments': ['Hi @jerryIsHere, thanks for reporting the issue. But are you sure this is a bug in HuggingFace **Datasets**?',
  '> Hi @jerryIsHere, thanks for reporting the issue. But are you sure th

In [35]:
# print the results
for i, score in enumerate(scores):
    print(f"{i+1}. Score: {score:.2f}")
    print(f"Comment: {retrieved_examples['text']}")
    print(f"URL: {retrieved_examples['html_url']}")
    print()

1. Score: 41.92
Comment: ['Tokenizer\'s normalization preprocessor cause misalignment in return_offsets_mapping for tokenizer classification task \n [This colab notebook](https://colab.research.google.com/drive/151gKyo0YIwnlznrOHst23oYH_a3mAe3Z?usp=sharing) implements a token classification input pipeline extending the logic from [this hugging example](https://huggingface.co/transformers/custom_datasets.html#tok-ner).\r\n\r\nThe pipeline works fine with most instance in different languages, but unfortunately, [the Japanese Kana ligature (a form of abbreviation? I don\'t know Japanese well)](https://en.wikipedia.org/wiki/Kana_ligature) break the alignment of `return_offsets_mapping`:\r\n![image](https://user-images.githubusercontent.com/50871412/122904371-db192700-d382-11eb-8917-1775db76db69.png)\r\n\r\nWithout the try catch block, it riase `ValueError: NumPy boolean array indexing assignment cannot assign 88 input values to the 87 output values where the mask is true`, example shown he

In [41]:
# let's collect the results in a dataframe
import pandas as pd
results_df = pd.DataFrame.from_dict(retrieved_examples)
results_df["scores"] = scores
results_df.sort_values(by="scores", ascending=False, inplace=True)

In [42]:
# print the results
for _, row in results_df.iterrows():
    print(f"Comment: {row.comments}")
    print(f"score: {row.scores:.2f}")
    print(f"Title: {row.title}")
    print(f"URL: {row.html_url}")
    print("+" * 50)
    print()

Comment: Hi! :)

I believe that should work unless `dataset_infos.json` isn't actually a dataset. For Hugging Face datasets, there is usually a file named `dataset_infos.json` which contains metadata about the dataset (eg. the dataset citation, license, description, etc). Can you double-check that `dataset_infos.json` isn't just metadata please?
score: 35.43
Title: load_dataset can't load local dataset,Unable to find ...
URL: https://github.com/huggingface/datasets/issues/4192
++++++++++++++++++++++++++++++++++++++++++++++++++

Comment: Hi @ahf876828330, 

As @stevhliu pointed out, the proper way to load a dataset is not trying to load its metadata file.

In your case, as the dataset script is local, you should better point to your local loading script:
```python
dataset = load_dataset("dataset/opus_books.py")
```

Please, feel free to re-open this issue if the previous code snippet does not work for you.
score: 35.43
Title: load_dataset can't load local dataset,Unable to find ...
URL: